<a href="https://colab.research.google.com/github/jeevan-nh/Google-collab-udemy-DL-course/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

##PART 1 - DATA PREPROCESSING

In [0]:
#importing the trainig set
dataset_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:,1:2].values

In [0]:
#Feature Scaling 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
training_set_scaled = scaler.fit_transform(training_set)

In [0]:
# Creating a data structure with 60 timesteps and 1 output
X_train,Y_train = [],[]
for i in range(60,1258):
  X_train.append(training_set_scaled[i-60:i,0])
  Y_train.append(training_set_scaled[i,0])
X_train,Y_train = np.array(X_train),np.array(Y_train)

In [0]:
#reshaping the array - from https://keras.io/layers/recurrent/ --> 3D tensor with shape (batch_size, timesteps, input_dim).
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

##PART 2 - BUILDING RNN

In [0]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [9]:
#Initialize the RNN
regressor = Sequential()

In [0]:
=